# Exercise 4  
Group Members: Xiaoyu Ji, Mengtong Guo, Yingchan Chu  
  
### 1 Constructing the matrix X

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import dok_matrix, coo_matrix
from scipy.sparse.linalg import lsqr

In [ ]:
def compute_sensor_size(M, Np=None):
    if Np is None:
        Np = int(np.ceil(np.sqrt(2) * M))
        Np += 1 if Np % 2 == 0 else 0
    return Np

In [ ]:
def construct_X(M, alphas, Np=None):
    Np = compute_sensor_size(M, Np)
    D = M * M
    No = len(alphas)
    # Create coordinate matrix for all pixels
    x, y = np.meshgrid(np.arange(M) - (M-1)/2, np.arange(M) - (M-1)/2)
    grid = np.vstack((x.flatten(), y.flatten()))

    i_indices = []
    j_indices = []
    weights = []

    # Process each projection angle
    for k, alpha in enumerate(alphas):
        cos_alpha, sin_alpha = np.cos(np.radians(alpha)), -np.sin(np.radians(alpha))
        p_vector = np.array([cos_alpha, sin_alpha])

        # Project coordinates onto the sensor line
        p = np.dot(p_vector, grid) + Np // 2

        # Compute the indices on the sensor
        i_floor = np.floor(p).astype(int)
        i_ceil = np.ceil(p).astype(int)

        # Ensure the indices are within the sensor array
        clip = (i_floor >= 0) & (i_ceil < Np)

        # Calculate the contributions to the nearest bins
        weight_floor = (i_ceil - p)[clip]
        weight_ceil = (p - i_floor)[clip]

        # Store indices and weights
        i_indices.extend([i_floor[clip] + k * Np, i_ceil[clip] + k * Np])
        j_indices.extend([np.arange(D)[clip], np.arange(D)[clip]])
        weights.extend([weight_floor, weight_ceil])

    # Create the sparse matrix X
    i = np.concatenate(i_indices)
    j = np.concatenate(j_indices)
    w = np.concatenate(weights)
    X = coo_matrix((w, (i, j)), shape=(No * Np, D), dtype=np.float32)

    return X

In [ ]:
X = np.load('hs_tomography/X_example.npy')

In [ ]:
# Construct X matrix and convert to dense format
X = construct_X(10, [-33, 1, 42]).todense()

# Set the graphics size and display the image
fig = plt.figure(figsize=(10, 4.5))
plt.imshow(X, interpolation='nearest')
plt.gray()
plt.axis('off')
plt.tight_layout()
plt.show()


<div style="color: green; font-weight: bold">Comment</div>
The function construct_X() is pretty similar to the provided solution but it looks like it produces some kind of X shaped artifact in the computed tomograms. This is due to some wrong calculations of their indices. They are using np.ceil(i) to calculate their second index but for integer numbers np.floor(i) == np.ceil(i). If they would replace i_ceil = np.ceil(p).astype(int) with i_ceil = i_floor + 1, their code works fine.

### 2 Recovering the image  
  
Firstly, we need to use the smaller version of the data for debugging the code.

In [ ]:
y_small = np.load('hs_tomography/y_77.npy')
alphas_small = np.load('hs_tomography/alphas_77.npy')

X_small = construct_X(77, alphas_small, 109).tocsc()

# Print shape and sparsity information
shape = f"Shape: {X_small.shape[0]} x {X_small.shape[1]}"
sparsity = f"Sparsity: {round(100 * (1 - X_small.nnz / np.prod(X_small.shape)), 2)} %"

print(f"{shape}\n{sparsity}\n")

Now we need to use `scipy.sparse.linalg.lsqr()` to obtain the least-squares solution to the problem.

In [ ]:
beta_small = lsqr(X_small, y_small, atol=1e-5, btol=1e-5)[0].reshape(77, 77)

fig, ax = plt.subplots(figsize=(4, 4))
cax = ax.imshow(beta_small, vmin=0, vmax=255, interpolation='nearest', cmap='gray')
ax.axis('off')
fig.tight_layout()

plt.show()

Now we implement it from a larger version of the data.

In [ ]:
y_large = np.load('hs_tomography/y_195.npy')
alphas_large = np.load('hs_tomography/alphas_195.npy')
X_large = construct_X(195, alphas_large, 275).tocsc()

# Print shape and sparsity information
shape = f"Shape: {X_large.shape[0]} x {X_large.shape[1]}"
sparsity = f"Sparsity: {round(100 * (1 - X_large.nnz / np.prod(X_large.shape)), 2)} %"

print(f"{shape}\n{sparsity}\n")

In [ ]:
beta_large = lsqr(X_large, y_large, atol=1e-5, btol=1e-5)[0].reshape(195, 195)

fig, ax = plt.subplots(figsize=(4, 4))
cax = ax.imshow(beta_large, vmin=0, vmax=255, interpolation='nearest', cmap='gray')
ax.axis('off')
fig.tight_layout()

plt.show()

We can see that the image is getting much clearer now, and we can also see something like a small stick in H.S.'s brain. The treatment now should be removing the stick by surgery.

<div style="color: green; font-weight: bold">Comment</div>
Both solutions are essentially the same. 

### 3 Minimizing the radiation dose

In [ ]:
Np = 275
y = np.load('hs_tomography/y_195.npy')
alphas = np.load('hs_tomography/alphas_195.npy')
                 

projection_angles = [4, 8, 16, 32, 48, 64]
fig, axes = plt.subplots(2, 3)
                 
for n, num_angles in enumerate(projection_angles):
    indices = [int(np.ceil(len(alphas) * p / num_angles)) for p in range(num_angles)]
    selected_alphas = alphas[indices]  
    selected_responses = []  


    for i in indices:
        selected_responses.extend(y[i*Np : (i+1)*Np])


    X = construct_X(195, selected_alphas, Np).tocsc()
    beta = lsqr(X, np.array(selected_responses), atol=1e-5, btol=1e-5)[0].reshape(195, 195)


    ax = axes.flat[n]
    ax.imshow(beta, vmin=0, vmax=255, interpolation='nearest')
    ax.set_title(f'{num_angles} projections')
    ax.axis('off')

plt.tight_layout()
plt.show()

For 32, the details are barely visible but it’s sufficient to suggest there's something abnormal. The 48 projections with more projections, the image clarity improves, making it easier to identify and examine the anomaly in more detail.And the image of 64 projections, clarity is significantly enhanced.

<div style="color: green; font-weight: bold">Comment</div>
Both solutions is exactly the same.